# Predicting EPL Football Match Winners Using Machine Learning
In this project, we'll predict match winners in the English Premier League (EPL) using machine learning. We'll be working with match data from the English Premier League. This data is from the 2020-2021 and 2021-2022 seasons.

In [23]:
import warnings
warnings.simplefilter(action='ignore')

# EDA Libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('fivethirtyeight')
%matplotlib inline

# ML Libraries
from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from yellowbrick.model_selection import FeatureImportances
from sklearn.feature_selection import SequentialFeatureSelector

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

**Load Dataset**

In [24]:
matches = pd.read_csv('matches.csv').drop(columns=['Unnamed: 0'])
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,1.3,64.0,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,0.1,67.0,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,3.8,0.1,80.0,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,0.8,61.0,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,0.4,63.0,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [49]:
matches['notes'].unique()

array([nan])

### Data Cleaning

In [25]:
matches['date'] = pd.to_datetime(matches['date'])
matches.set_index('date', inplace=True)

In [26]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1389 entries, 2021-08-15 to 2021-05-23
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time          1389 non-null   object 
 1   comp          1389 non-null   object 
 2   round         1389 non-null   object 
 3   day           1389 non-null   object 
 4   venue         1389 non-null   object 
 5   result        1389 non-null   object 
 6   gf            1389 non-null   float64
 7   ga            1389 non-null   float64
 8   opponent      1389 non-null   object 
 9   xg            1389 non-null   float64
 10  xga           1389 non-null   float64
 11  poss          1389 non-null   float64
 12  attendance    693 non-null    float64
 13  captain       1389 non-null   object 
 14  formation     1389 non-null   object 
 15  referee       1389 non-null   object 
 16  match report  1389 non-null   object 
 17  notes         0 non-null      float64
 18  sh        

### Investigating Missing Data
In the English Premier League, there are 20 teams, and each team plays 38 matches. We have data for two seasons. So we should have 2 * 20 * 38 matches, or 1520. Three teams are relegated each season to a lower league, and three are promoted. Given the relegations & promotions that happened at the end of the 2020-2021 season, we should have 6 teams with 38 matches and 17 teams with 76 matches.

Let's determine if there's any missing data.

**Match Count**

There are 1389 matches in our dataset. With our expected total at 1520, this means we have a deficit of 131 matches not reflected in the dataset.

In [31]:
matches.index.year.unique()

Int64Index([2021, 2022, 2020], dtype='int64', name='date')

In [32]:
matches.shape

(1389, 26)

**Team & Match Count**

There are 23 teams. 7 teams have 38 matches or under. 3 of those 7 have match totals under 38. 16 teams have matches in excess of 70, but none of them have 76 total matches.

In [33]:
matches['team'].value_counts()

Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Liverpool                   38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   34
Watford                     33
Norwich City                33
Name: team, dtype: int64

In [34]:
matches['team'].value_counts().loc[matches['team'].value_counts() <=38]

Liverpool               38
Fulham                  38
West Bromwich Albion    38
Sheffield United        38
Brentford               34
Watford                 33
Norwich City            33
Name: team, dtype: int64

Liverpool seems to be included in the relegated teams list. Upon review, it seems that the dataset only contains records for the 2020/2021 season. We'll have to collect and include the records for the 2021/2022 season.

In [36]:
matches.loc[matches.team == 'Liverpool'].index.year.unique()

Int64Index([2020, 2021], dtype='int64', name='date')

In [37]:
matches['team'].value_counts().loc[matches['team'].value_counts() > 38]

Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Name: team, dtype: int64

**Missing Dates**

In [38]:
missing_dates_20 = pd.date_range(start='2020-09-12', end='2021-05-23').difference(matches.index).tolist()
missing_dates_20 = pd.Series(missing_dates_20)
missing_dates_20.shape

(119,)

In [ ]:
matches['date'].min()

In [39]:
missing_dates_21 = pd.date_range(start='2021-08-14', end='2022-05-22').difference(matches.index).tolist()
missing_dates_21 = pd.Series(missing_dates_21)
missing_dates_21.shape

(174,)

In [44]:
matches_20 = matches.loc[(matches.index > '2020-09-11') & (matches.index <= '2021-05-23')].sort_index()
matches_20.head()

,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-12,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,1.0,Crystal Palace,0.8,0.7,69.0,NaN,James Ward-Prowse,4-4-2,Jonathan Moss,Match Report,NaN,9.0,5.0,15.6,2.0,0.0,0.0,2021,Southampton
2020-09-12,17:30,Premier League,Matchweek 1,Sat,Away,L,3.0,4.0,Liverpool,0.6,3.3,50.0,NaN,Luke Ayling,4-1-4-1,Michael Oliver,Match Report,NaN,6.0,3.0,17.5,1.0,0.0,0.0,2021,Leeds United
2020-09-12,17:30,Premier League,Matchweek 1,Sat,Home,W,4.0,3.0,Leeds United,3.3,0.6,50.0,NaN,Jordan Henderson,4-3-3,Michael Oliver,Match Report,NaN,20.0,4.0,17.0,0.0,2.0,2.0,2021,Liverpool
2020-09-12,12:30,Premier League,Matchweek 1,Sat,Home,L,0.0,3.0,Arsenal,0.2,1.8,45.0,NaN,Tom Cairney,4-2-3-1,Chris Kavanagh,Match Report,NaN,5.0,2.0,26.0,0.0,0.0,0.0,2021,Fulham
2020-09-12,12:30,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Fulham,1.8,0.2,55.0,NaN,Pierre-Emerick Aubameyang,3-4-3,Chris Kavanagh,Match Report,NaN,13.0,5.0,13.6,2.0,0.0,0.0,2021,Arsenal


In [45]:
matches_21 = matches.loc[(matches.index > '2021-08-14') & (matches.index <= '2022-05-23')].sort_index()
matches_21.head()

,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,1.3,64.0,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2021-08-15,14:00,Premier League,Matchweek 1,Sun,Home,L,2.0,4.0,West Ham,1.5,2.3,47.0,50673.0,Federico Fernández,3-5-2,Martin Atkinson,Match Report,NaN,17.0,3.0,15.3,0.0,0.0,0.0,2022,Newcastle United
2021-08-15,16:30,Premier League,Matchweek 1,Sun,Home,W,1.0,0.0,Manchester City,1.3,1.9,36.0,58262.0,Hugo Lloris,4-3-3,Anthony Taylor,Match Report,NaN,14.0,2.0,18.4,0.0,0.0,0.0,2022,Tottenham Hotspur
2021-08-15,14:00,Premier League,Matchweek 1,Sun,Away,W,4.0,2.0,Newcastle Utd,2.3,1.5,53.0,50673.0,Declan Rice,4-2-3-1,Martin Atkinson,Match Report,NaN,17.0,7.0,16.1,1.0,0.0,1.0,2022,West Ham United
2021-08-21,17:30,Premier League,Matchweek 2,Sat,Away,L,0.0,2.0,Brighton,0.9,1.5,43.0,29485.0,Tom Cleverley,4-1-4-1,Anthony Taylor,Match Report,NaN,10.0,1.0,18.0,0.0,0.0,0.0,2022,Watford


In [46]:
matches_20.loc[matches_20['team'] == 'Manchester United'].sort_index()

,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-19,17:30,Premier League,Matchweek 2,Sat,Home,L,1.0,3.0,Crystal Palace,1.1,1.6,75.0,NaN,Harry Maguire,4-2-3-1,Martin Atkinson,Match Report,NaN,17.0,4.0,18.9,1.0,0.0,0.0,2021,Manchester United
2020-09-26,12:30,Premier League,Matchweek 3,Sat,Away,W,3.0,2.0,Brighton,1.1,2.4,46.0,NaN,Harry Maguire,4-2-3-1,Chris Kavanagh,Match Report,NaN,6.0,2.0,17.8,0.0,1.0,1.0,2021,Manchester United
2020-10-04,16:30,Premier League,Matchweek 4,Sun,Home,L,1.0,6.0,Tottenham,0.9,3.8,38.0,NaN,Harry Maguire,4-2-3-1,Anthony Taylor,Match Report,NaN,4.0,1.0,22.7,0.0,1.0,1.0,2021,Manchester United
2020-10-17,20:00,Premier League,Matchweek 5,Sat,Away,W,4.0,1.0,Newcastle Utd,2.7,0.5,63.0,NaN,Harry Maguire,4-2-3-1,Craig Pawson,Match Report,NaN,27.0,11.0,19.5,0.0,0.0,1.0,2021,Manchester United
2020-10-24,17:30,Premier League,Matchweek 6,Sat,Home,D,0.0,0.0,Chelsea,0.7,0.2,50.0,NaN,Harry Maguire,4-2-3-1,Martin Atkinson,Match Report,NaN,14.0,4.0,16.9,0.0,0.0,0.0,2021,Manchester United
2020-11-01,16:30,Premier League,Matchweek 7,Sun,Home,L,0.0,1.0,Arsenal,0.3,1.1,53.0,NaN,Harry Maguire,4-4-2◆,Mike Dean,Match Report,NaN,9.0,2.0,18.5,0.0,0.0,0.0,2021,Manchester United
2020-11-07,12:30,Premier League,Matchweek 8,Sat,Away,W,3.0,1.0,Everton,1.6,0.6,45.0,NaN,Harry Maguire,4-2-3-1,Paul Tierney,Match Report,NaN,13.0,5.0,20.8,2.0,0.0,0.0,2021,Manchester United
2020-11-21,20:00,Premier League,Matchweek 9,Sat,Home,W,1.0,0.0,West Brom,2.5,0.6,62.0,NaN,Harry Maguire,4-2-3-1,David Coote,Match Report,NaN,16.0,6.0,18.4,1.0,1.0,1.0,2021,Manchester United
2020-11-29,14:00,Premier League,Matchweek 10,Sun,Away,W,3.0,2.0,Southampton,2.3,0.4,50.0,NaN,Harry Maguire,4-4-2◆,Jonathan Moss,Match Report,NaN,15.0,6.0,16.0,0.0,0.0,0.0,2021,Manchester United


In [47]:
matches_21.loc[matches_21['team'] == 'Manchester United'].sort_index()

,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-22,14:00,Premier League,Matchweek 2,Sun,Away,D,1.0,1.0,Southampton,1.4,0.8,62.0,32000.0,Harry Maguire,4-2-3-1,Craig Pawson,Match Report,NaN,15.0,3.0,13.9,1.0,0.0,0.0,2022,Manchester United
2021-08-29,16:30,Premier League,Matchweek 3,Sun,Away,W,1.0,0.0,Wolves,0.6,1.8,56.0,30621.0,Harry Maguire,4-2-3-1,Mike Dean,Match Report,NaN,10.0,3.0,18.3,1.0,0.0,0.0,2022,Manchester United
2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,1.0,Newcastle Utd,2.1,0.7,64.0,72732.0,Harry Maguire,4-2-3-1,Anthony Taylor,Match Report,NaN,21.0,6.0,20.3,0.0,0.0,0.0,2022,Manchester United
2021-09-19,14:00,Premier League,Matchweek 5,Sun,Away,W,2.0,1.0,West Ham,1.8,1.6,61.0,60000.0,Harry Maguire,4-2-3-1,Martin Atkinson,Match Report,NaN,17.0,10.0,16.4,0.0,0.0,0.0,2022,Manchester United
2021-09-25,12:30,Premier League,Matchweek 6,Sat,Home,L,0.0,1.0,Aston Villa,2.3,1.2,58.0,72922.0,Harry Maguire,4-2-3-1,Mike Dean,Match Report,NaN,27.0,4.0,18.9,2.0,0.0,1.0,2022,Manchester United
2021-10-02,12:30,Premier League,Matchweek 7,Sat,Home,D,1.0,1.0,Everton,0.9,1.1,72.0,73128.0,Bruno Fernandes,4-2-3-1,Michael Oliver,Match Report,NaN,13.0,6.0,14.7,0.0,0.0,0.0,2022,Manchester United
2021-10-16,15:00,Premier League,Matchweek 8,Sat,Away,L,2.0,4.0,Leicester City,1.4,3.1,51.0,32219.0,Harry Maguire,4-2-3-1,Craig Pawson,Match Report,NaN,18.0,6.0,19.7,0.0,0.0,0.0,2022,Manchester United
2021-10-24,16:30,Premier League,Matchweek 9,Sun,Home,L,0.0,5.0,Liverpool,1.5,3.8,37.0,73088.0,Harry Maguire,4-2-3-1,Anthony Taylor,Match Report,NaN,12.0,3.0,15.9,0.0,0.0,0.0,2022,Manchester United
2021-10-30,17:30,Premier League,Matchweek 10,Sat,Away,W,3.0,0.0,Tottenham,1.2,0.7,42.0,60356.0,Harry Maguire,3-4-1-2,Stuart Attwell,Match Report,NaN,10.0,4.0,16.7,0.0,0.0,0.0,2022,Manchester United


Determine how many matches we have data for. Does it match the number we should have?
How many rows are missing due to the data being scraped partway through the season? You can determine this by looking at the date column.
Are there any teams that are missing more data than you'd expect?

In [50]:
matches.head(200)

,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,1.3,64.0,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,0.1,67.0,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,3.8,0.1,80.0,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,0.8,61.0,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,0.4,63.0,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1.0,0.0,Chelsea,1.7,0.3,60.0,40036.0,Rúben Dias,4-3-3,Michael Oliver,Match Report,NaN,15.0,3.0,16.5,0.0,0.0,0.0,2022,Manchester City
2021-10-03,16:30,Premier League,Matchweek 7,Sun,Away,D,2.0,2.0,Liverpool,1.2,1.0,51.0,53102.0,Rúben Dias,4-3-3,Paul Tierney,Match Report,NaN,12.0,3.0,16.8,1.0,0.0,0.0,2022,Manchester City
2021-10-16,15:00,Premier League,Matchweek 8,Sat,Home,W,2.0,0.0,Burnley,1.9,1.0,69.0,52711.0,Kevin De Bruyne,4-3-3,Martin Atkinson,Match Report,NaN,16.0,6.0,14.6,0.0,0.0,0.0,2022,Manchester City
2021-10-23,17:30,Premier League,Matchweek 9,Sat,Away,W,4.0,1.0,Brighton,3.9,1.3,52.0,31215.0,İlkay Gündoğan,4-3-3,Kevin Friend,Match Report,NaN,23.0,13.0,15.7,1.0,0.0,0.0,2022,Manchester City
